In [0]:
from pyspark.sql import functions as F

predictions_table = "nlp_dev.ml.fasttext_predictions"

df_pred = spark.table(predictions_table)

# If you have an ingest or prediction timestamp, use it; if not, we fake it for now.
# Example: assume there is a 'prediction_time' or fallback to 'ingest_date'


In [0]:
df_stats = (
    df_pred
      .withColumn("prediction_date", F.to_date("ingest_date"))  # adjust column name if needed
      .groupBy("prediction_date", "label", "prediction")
      .agg(
          F.count("*").alias("row_count")
      )
)

display(df_stats)


In [0]:
stats_path = (
    "abfss://nlp@nlplakeadls001.dfs.core.windows.net/"
    "agents/prediction_stats"
)

(
    df_stats.write
      .format("delta")
      .mode("overwrite")   # or 'append' if you later do incremental
      .save(stats_path)
)


In [0]:
%sql
CREATE TABLE IF NOT EXISTS nlp_dev.agents.prediction_stats
USING DELTA
LOCATION 'abfss://nlp@nlplakeadls001.dfs.core.windows.net/agents/prediction_stats';
